In [1]:
import pandas as pd

from pandarallel import pandarallel
from freeholdforecast.tasks.etl_ml_task import ETL_ML_Task

pd.options.display.max_columns = 100

task = ETL_ML_Task()


2022-09-09 18:13:12 INFO ETL_ML_Task Initializing task for NCME with run date 2016-09-01
2022-09-09 18:13:12 INFO ETL_ML_Task Train years: 5
2022-09-09 18:13:12 INFO ETL_ML_Task Train dates: 2011-08-01 to 2016-08-01
2022-09-09 18:13:12 INFO ETL_ML_Task Test dates: 2016-09-01 to 2016-09-01


In [2]:
task._get_df_raw_encoded()
# task.df_raw


2022-09-09 18:13:12 INFO ETL_ML_Task Retrieving and encoding data
2022-09-09 18:13:12 INFO ETL_ML_Task Loading existing raw data
2022-09-09 18:13:16 INFO ETL_ML_Task Total parcels: 315690
2022-09-09 18:13:16 INFO ETL_ML_Task Total sales: 763441
2022-09-09 18:13:16 INFO ETL_ML_Task Loading existing encoded data


In [3]:
task._get_df_prepared()


2022-09-09 18:13:16 INFO ETL_ML_Task Loading existing prepared data
2022-09-09 18:13:17 INFO ETL_ML_Task Splitting data


In [4]:
task._train_models()


2022-09-09 18:13:17 INFO ETL_ML_Task Total labels: 4
2022-09-09 18:13:17 INFO ETL_ML_Task Jobs per label: 2
2022-09-09 18:13:17 INFO ETL_ML_Task Total fit minutes: 60
2022-09-09 18:13:17 INFO ETL_ML_Task Job fit minutes: 30
2022-09-09 18:13:17 INFO ETL_ML_Task Job memory limit (MB): 3072
2022-09-09 18:13:17 INFO ETL_ML_Task Initialize AutoML for transfer_in_3_months
2022-09-09 18:13:17 INFO ETL_ML_Task Train labels: 14741/747499 (1.97%)
2022-09-09 18:13:18 INFO ETL_ML_Task Train resamped labels: 14741/113014 (13.04%)
2022-09-09 18:13:18 INFO ETL_ML_Task Test labels: 240/12907 (1.86%)
2022-09-09 18:13:18 INFO ETL_ML_Task Initialize AutoML for transfer_in_6_months
2022-09-09 18:13:18 INFO ETL_ML_Task Train labels: 28857/747499 (3.86%)
2022-09-09 18:13:18 INFO ETL_ML_Task Train resamped labels: 28857/221237 (13.04%)
2022-09-09 18:13:18 INFO ETL_ML_Task Test labels: 486/12907 (3.77%)
2022-09-09 18:13:18 INFO ETL_ML_Task Initialize AutoML for transfer_in_12_months
2022-09-09 18:13:18 INFO E

/home/vscode/.local/share/virtualenvs/freehold-forecast-zrIN4e5_/lib/python3.8/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44521 instead
  warnings.warn(
/home/vscode/.local/share/virtualenvs/freehold-forecast-zrIN4e5_/lib/python3.8/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39071 instead
  warnings.warn(


[WARNING] [2022-09-09 18:13:23,211:Client-AutoML(1):160adbc7-306b-11ed-96f1-0242ac110002] Dataset too large for memory limit 3072MB, reducing the precision from float64 to <class 'numpy.float32'>
[WARNING] [2022-09-09 18:13:23,312:Client-AutoML(1):160adbc7-306b-11ed-96f1-0242ac110002] Dataset too large for memory limit 3072MB, reducing number of samples from 747499 to 592137.
[WARNING] [2022-09-09 18:13:23,607:Client-AutoML(1):15f2cf21-306b-11ed-96ee-0242ac110002] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
[WARNING] [2022-09-09 18:13:23,983:Client-AutoML(1):15f3a07a-306b-11ed-96ef-0242ac110002] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
[WARNING] [2022-09-09 18:13:24,069:Client-AutoML(1):15f44502-306b-11ed-96f0-0242ac110002] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.


/home/vscode/.local/share/virtualenvs/freehold-forecast-zrIN4e5_/lib/python3.8/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38663 instead
  warnings.warn(


[WARNING] [2022-09-09 18:13:26,757:Client-AutoML(1):160adbc7-306b-11ed-96f1-0242ac110002] Capping the per_run_time_limit to 1799.0 to have time for a least 2 models in each process.
2022-09-09 19:13:22 INFO ETL_ML_Task Saving model for transfer_in_3_months
2022-09-09 19:13:22 INFO ETL_ML_Task Saving model for transfer_in_6_months
2022-09-09 19:13:23 INFO ETL_ML_Task Saving model for transfer_in_12_months
2022-09-09 19:13:25 INFO ETL_ML_Task Saving model for transfer_in_24_months


/home/vscode/.local/share/virtualenvs/freehold-forecast-zrIN4e5_/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/home/vscode/.local/share/virtualenvs/freehold-forecast-zrIN4e5_/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/home/vscode/.local/share/virtualenvs/freehold-forecast-zrIN4e5_/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/home/vscode/.local/share/virtualenvs/freehold-forecast-zrIN4e5_/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


2022-09-09 19:16:02 INFO ETL_ML_Task Model metrics for transfer_in_3_months
2022-09-09 19:16:05 INFO ETL_ML_Task auto-sklearn results:
  Dataset name: 15f2cf21-306b-11ed-96ee-0242ac110002
  Metric: roc_auc
  Best validation score: 0.960196
  Number of target algorithm runs: 56
  Number of successful target algorithm runs: 49
  Number of crashed target algorithm runs: 1
  Number of target algorithms that exceeded the time limit: 2
  Number of target algorithms that exceeded the memory limit: 4

2022-09-09 19:16:08 INFO ETL_ML_Task Pred labels: 179/12907 (1.39%)
2022-09-09 19:16:08 INFO ETL_ML_Task Average precision: 0.43
2022-09-09 19:16:08 INFO ETL_ML_Task Precision: 0.75
2022-09-09 19:16:08 INFO ETL_ML_Task ROC AUC: 0.78
2022-09-09 19:16:08 INFO ETL_ML_Task Classification report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99     12667
           1       0.75      0.56      0.64       240

    accuracy                           0.99 